In [4]:
import thinkdsp
import numpy as np

def detect_one_or_zero(segment, qfactor=.99, lowfreq=500, highfreq=1500):
    if len(segment.ys) < 3:
        print("too short")
        return
    segment2 = segment
    x = segment2.ys
    y = [ 0 for i in xrange(len(segment2.ys))]
    q = qfactor
    wo = (lowfreq * np.pi * 2) / segment2.framerate

    y[0] = x[0] + 2 * q - 2

    y[1] = x[1] + 2 * q * np.cos(wo) * y[0] - 2 * np.cos(wo) * x[0]

    for i in range(2, len(segment2.ys)):
        y[i] = x[i] + 2*q*np.cos(wo) * y[i-1] - q*q*y[i - 2] - 2 * np.cos(wo) * x[i-1] + x[i - 2]
    lowwave = thinkdsp.Wave(y, segment2.framerate)
    
    wo = (highfreq * np.pi * 2) / segment2.framerate
    
    y = [ 0 for i in xrange(len(segment2.ys))]
    
    y[0] = x[0] + 2 * q - 2

    y[1] = x[1] + 2 * q * np.cos(wo) * y[0] - 2 * np.cos(wo) * x[0]

    for i in range(2, len(segment2.ys)):
        y[i] = x[i] + 2*q*np.cos(wo) * y[i-1] - q*q*y[i - 2] - 2 * np.cos(wo) * x[i-1] + x[i - 2]
    highwave = thinkdsp.Wave(y, segment2.framerate)
    
    lowAverage = np.average(np.absolute(lowwave.ys))
    highAverage = np.average(np.absolute(highwave.ys))
    if lowAverage > highAverage and lowAverage > 1000: #tweak constant
        return "1"
    elif highAverage > lowAverage and highAverage > 1000: #tweak me
        return "0"
def find_start_and_end(xf, threshold = 2000): 
    import numpy as np    
    start_idx = -1
 
    for k in range(0, len(xf)):
        if(np.abs(xf[k])) > threshold:
            start_idx = k
            break

    if(start_idx  < 0):
        print("Unable to detect start of transmission")
        return -1
    
    end_idx = -1
    
    for k in range(0, len(xf)):
        if(np.abs(xf[len(xf)-k-1])) > threshold:
            end_idx = len(xf)-k-1
            break

    if(end_idx < 0):
        print("Unable to detect end of transmission")
        return -1

    return start_idx, end_idx

def process_wav(filename, symbolLength=.5):
    wave = thinkdsp.read_wave(filename)
    (start, end) = find_start_and_end(wave.ys)
    subwave = thinkdsp.Wave(wave.ys[start:end], wave.framerate)
    for i in range(0, int(round(subwave.duration / symbolLength))):
        print(detect_one_or_zero(subwave.segment(i * symbolLength, symbolLength)))

In [5]:
process_wav("receivedfile.wav")

NameError: global name 'windowSize' is not defined